In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('FINAL.csv')

In [3]:
df['venue'].unique(),df['bowling_team'].unique(),df['over_type'].unique()

(array(['Chinnaswamy Stadium(Banglore)', 'PCA Stadium Mohali',
        'Feroz shah kotla(Delhi)', 'Wankhede Stadium(Mumbai)',
        'Eden Gardens', 'Sawai Mansingh Stadium',
        'Rajiv Gandhi stadium(Hyderbad)', 'Chepauk Stadium(Chennai)',
        'Dr DY Patil Sports Academy', 'Newlands', "St George's Park",
        'Kingsmead', 'SuperSport Park', 'Buffalo Park',
        'New Wanderers Stadium', 'De Beers Diamond Oval',
        'OUTsurance Oval', 'CCI(Mumbai)', 'Motera Stadium(Ahemdabad)',
        'Barabati Stadium', 'Brabourne Stadium, Mumbai',
        'Vidarbha Cricket Association Stadium, Jamtha',
        'Himachal Pradesh Cricket Association Stadium', 'Nehru Stadium',
        'Holkar Cricket Stadium',
        'Dr. Y.S. Rajasekhara Reddy Cricket Stadium(vizag)',
        'Pune stadium', 'Shaheed Veer Narayan Singh International Stadium',
        'JSCA International Stadium Complex', 'Sheikh Zayed Stadium',
        'Sharjah Cricket Stadium', 'Dubai International Cricket Stadium'

In [8]:
df=pd.read_csv('FINAL.csv')
current_venue=input('Enter the venue: ')
current_phase=input('Enter the phase: ')
current_opposition=input('Enter the opposition')


def balls_per_dismissal(balls, dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1 
    
def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1 


df = df[df.venue == current_venue]
df = df[df.over_type == current_phase]
df = df[df.bowling_team == current_opposition]

#df.reset_index(inplace = True, drop = True)

df['isDot'] = df['runs_off_bat'].apply(lambda x: 1 if x == 0 else 0)
df['isOne'] = df['runs_off_bat'].apply(lambda x: 1 if x == 1 else 0)
df['isTwo'] = df['runs_off_bat'].apply(lambda x: 1 if x == 2 else 0)
df['isThree'] = df['runs_off_bat'].apply(lambda x: 1 if x == 3 else 0)
df['isFour'] = df['runs_off_bat'].apply(lambda x: 1 if x == 4 else 0)
df['isSix'] = df['runs_off_bat'].apply(lambda x: 1 if x == 6 else 0)

runs = pd.DataFrame(df.groupby(['striker', 'match_id'])['runs_off_bat'].sum().reset_index()).groupby(['striker'])['runs_off_bat'].sum().reset_index().rename(columns={'runs_off_bat':'runs'})
innings = pd.DataFrame(df.groupby(['striker'])['match_id'].apply(lambda x: len(list(np.unique(x)))).reset_index()).rename(columns = {'match_id':'innings'})
balls = pd.DataFrame(df.groupby(['striker'])['match_id'].count()).reset_index().rename(columns = {'match_id':'balls'})
dismissals = pd.DataFrame(df.groupby(['striker'])['player_dismissed'].count()).reset_index().rename(columns = {'player_dismissed':'dismissals'})

dots = pd.DataFrame(df.groupby(['striker'])['isDot'].sum()).reset_index().rename(columns = {'isDot':'dots'})
ones = pd.DataFrame(df.groupby(['striker'])['isOne'].sum()).reset_index().rename(columns = {'isOne':'ones'})
twos = pd.DataFrame(df.groupby(['striker'])['isTwo'].sum()).reset_index().rename(columns = {'isTwo':'twos'})
threes = pd.DataFrame(df.groupby(['striker'])['isThree'].sum()).reset_index().rename(columns = {'isThree':'threes'})
fours = pd.DataFrame(df.groupby(['striker'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'fours'})
sixes = pd.DataFrame(df.groupby(['striker'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'sixes'})

df = pd.merge(innings, runs, on = 'striker').merge(balls, on = 'striker').merge(dismissals, on = 'striker').merge(dots, on = 'striker').merge(ones, on = 'striker').merge(twos, on = 'striker').merge(threes, on = 'striker').merge(fours, on = 'striker').merge(sixes, on = 'striker')

#StrikeRate
df['SR'] = df.apply(lambda x: 100*(x['runs']/x['balls']), axis = 1)

#runs per innings
df['RPI'] = df.apply(lambda x: x['runs']/x['innings'], axis = 1)

#balls per dismissals
df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['balls'], x['dismissals']), axis = 1)

#balls per boundary
df['BPB'] = df.apply(lambda x: balls_per_boundary(x['balls'], (x['fours'] + x['sixes'])), axis = 1)

df['dot_percentage'] = df.apply(lambda x: x['dots']/x['balls'], axis = 1)
wt_sr, wt_rpi, wt_bpd, wt_dot_percentage = 0.13, 0.27, 0.16, 0.45
df = df[df.innings >= 2 ]

df['calc_SR'] = df['SR'].apply(lambda x: x*x) 
df['calc_RPI'] = df['RPI'].apply(lambda x: x*x) 
df['calc_BPD'] = df['BPD'].apply(lambda x: x*x) 
df['calc_dot_percentage'] = df['dot_percentage'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(df[['calc_SR','calc_RPI', 'calc_BPD', 'calc_dot_percentage']].sum(axis = 0))

df['calc_SR'] = df['calc_SR'].apply(lambda x: x/sq_sr) 
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x/sq_rpi) 
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x/sq_bpd) 
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x/sq_dot_percentage)

df['calc_SR'] = df['calc_SR'].apply(lambda x: x*wt_sr) 
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x*wt_rpi) 
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x*wt_bpd) 
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x*wt_dot_percentage)

best_sr, worst_sr = max(df['calc_SR']), min(df['calc_SR'])
best_rpi, worst_rpi = max(df['calc_RPI']), min(df['calc_RPI'])
best_bpd, worst_bpd = max(df['calc_BPD']), min(df['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(df['calc_dot_percentage']), max(df['calc_dot_percentage'])

df['dev_best_SR'] = df['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr)) 
df['dev_best_RPI'] = df['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi)) 
df['dev_best_BPD'] = df['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd)) 
df['dev_best_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

df['dev_best_sqrt'] = df.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_dot_percentage'], axis = 1) 

df['dev_worst_SR'] = df['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr)) 
df['dev_worst_RPI'] = df['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi)) 
df['dev_worst_BPD'] = df['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd)) 
df['dev_worst_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

df['dev_worst_sqrt'] = df.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_dot_percentage'], axis = 1) 
df['score'] = df.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)
df[[ 'striker','innings', 'runs', 'balls', 'dismissals', 'dot_percentage', 'score']].sort_values(['score'], ascending = False).reset_index(drop = True).head(25)

Enter the venue: Eden Gardens
Enter the phase: Death overs
Enter the oppositionKolkata Knight Riders


,striker,innings,runs,balls,dismissals,dot_percentage,score
0,Mandeep Singh,2,60,25,25,0.120000,0.967631
1,AR Patel,2,21,8,8,0.125000,0.917505
2,CH Gayle,3,78,35,35,0.342857,0.871315
3,BCJ Cutting,2,32,14,14,0.357143,0.830243
4,V Kohli,3,78,36,36,0.194444,0.827484
5,AT Rayudu,4,83,40,40,0.225000,0.704126
6,JC Archer,2,28,14,14,0.214286,0.576509
7,MS Dhoni,6,117,60,60,0.266667,0.571179
8,RG Sharma,5,92,47,47,0.276596,0.568733
9,HH Pandya,3,38,19,19,0.421053,0.567973
